In [13]:
from dandi.dandiapi import DandiAPIClient
from tqdm.notebook import tqdm_notebook

In [4]:
client = DandiAPIClient()
dandisets = list(client.get_dandisets())

In [15]:
nwb_dandisets = []

for dandiset in tqdm(dandisets):
    metadata = dandiset.get_raw_metadata()
    if any(
        x['identifier'] == 'RRID:SCR_015242'
        for x in dandiset.get_raw_metadata()['assetsSummary'].get('dataStandard', {})
    ):
        nwb_dandisets.append(dandiset)


100%|████████████████████████████████████████████████████████████████████████████| 210/210 [02:01<00:00,  1.73it/s]


In [40]:
len(nwb_dandisets)

119

In [44]:
mouse_nwb_dandisets = []

for dandiset in tqdm(nwb_dandisets):
    metadata = dandiset.get_raw_metadata()
    if any("mouse" in x["name"] for x in metadata["assetsSummary"].get("species", [])):
        mouse_nwb_dandisets.append(dandiset)
len(mouse_nwb_dandisets)

100%|████████████████████████████████████████████████████████████████████████████| 119/119 [00:29<00:00,  3.99it/s]


58

# Session-level search: NWB sessions with male mice

The `DandiAPIClient` can also be used to search for metadata of individual sessions. For example, you can find sessions with a male mouse. First, it's a good idea to filter down to dandisets that contain mice, then for each session test if the subject is a mouse and of it is male.

In [68]:
male_mouse_nwb_sessions = []

for dandiset in tqdm_notebook(nwb_dandisets, position=0):
    assets = list(dandiset.get_assets()) # each asset represents an individual NWB file (session)
    for asset in tqdm_notebook(assets, position=1, leave=False, desc=dandiset.identifier):
        asset_metadata = asset.get_metadata()
        subjects = asset_metadata.wasAttributedTo
        # there is usually only 1 subject per session but we iterate over the list in case there are more
        for subject in subjects:
            # first check that the metadata object exists, then check its value
            if (
                subject.species and "mouse" in subject.species.name \
                and subject.sex and subject.sex.name == "Male"
            ):
                male_mouse_nwb_sessions.append(asset)
            
        

  0%|          | 0/119 [00:00<?, ?it/s]

000003:   0%|          | 0/101 [00:00<?, ?it/s]

000004:   0%|          | 0/87 [00:00<?, ?it/s]

000005:   0%|          | 0/148 [00:00<?, ?it/s]

000006:   0%|          | 0/53 [00:00<?, ?it/s]

...

000027:   0%|          | 0/1 [00:00<?, ?it/s]

000028:   0%|          | 0/3 [00:00<?, ?it/s]

000029:   0%|          | 0/4 [00:00<?, ?it/s]

000034:   0%|          | 0/6 [00:00<?, ?it/s]

TypeError: 'NoneType' object is not iterable

'000003'

In [38]:
asset_list = []
for dandiset in dandisets:
    if dandiset.identifier not in identifiers:
        continue
    for asset in tqdm(list(dandiset.get_assets())):
        asset_metadata = asset.get_metadata()
        sex = asset_metadata.wasAttributedTo[0].sex
        if sex and sex.name == "Male":
            asset_list.append(asset)

119

In [32]:
metadata["assetsSummary"].get("species", [])


[{'name': 'House mouse',
  'schemaKey': 'SpeciesType',
  'identifier': 'http://purl.obolibrary.org/obo/NCBITaxon_10090'}]

In [11]:
is_nwb(metadata)

True